In [1]:
!pip install azure-ai-ml

In [2]:
!pip install azure-identity

In [3]:
pip install azureml-train-automl

  Using cached azureml_train_automl-1.33.0-py3-none-any.whl (3.4 kB)
INFO: pip is looking at multiple versions of azureml-train-automl to determine which version is compatible with other requirements. This could take a while.
  Using cached azureml_train_automl-1.32.0-py3-none-any.whl (3.4 kB)
  Using cached azureml_train_automl-1.31.0-py3-none-any.whl (3.4 kB)
  Using cached azureml_train_automl-1.30.0-py3-none-any.whl (3.4 kB)
  Using cached azureml_train_automl-1.29.0-py3-none-any.whl (3.4 kB)
  Using cached azureml_train_automl-1.28.0-py3-none-any.whl (3.4 kB)
  Using cached azureml_train_automl-1.27.0-py3-none-any.whl (3.4 kB)
  Using cached azureml_train_automl-1.26.0-py3-none-any.whl (3.4 kB)
INFO: pip is looking at multiple versions of azureml-train-automl to determine which version is compatible with other requirements. This could take a while.
  Using cached azureml_train_automl-1.25.0-py3-none-any.whl (3.4 kB)
  Using cached azureml_train_automl-1.24.0-py3-none-any.whl (3.4 

In [4]:
import pandas as pd
import os
import logging
from azure.identity import DefaultAzureCredential
from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azure.ai.ml import automl, Input, MLClient
from pprint import pprint
from azure.ai.ml.entities import ResourceConfiguration
from sklearn.model_selection import train_test_split
from azure.ai.ml.entities import AmlCompute
from azure.core.exceptions import ResourceExistsError, ResourceNotFoundError
import azureml.core
import mltable
from mltable import MLTableHeaders, MLTableFileEncoding, DataType

# <mark>**CARGA DE CREDENCIALES**</mark>

In [5]:
credential = DefaultAzureCredential()
ml_client = MLClient.from_config(credential)

Found the config file in: /config.json


## <mark>**PROCEDER A DESCARGAR EL DATASET YA FILTRADO PARA EL ENTRENAMIENTO**</mark>

SE PROCEDIO A REALIZAR TAMBIEN LA DIVISION DEL DATA PARA LA PARTE DEL ENTRENAMIENTO EN UN 80% Y LA VALIDACION EN 20%, POSTERIORMENTE SE GUARDO LOS DATAS CREADOS PARA SU UTILIZACION EN EL ENTRENAMIENTO DEL MODELO. 

In [6]:
df = pd.read_csv('./data/bebedores_alcohol_compri.csv')
train, test = train_test_split(df, test_size=0.2, random_state=42)
train. to_csv ( './data/training-table/train_data.csv', index=False, encoding='utf-8')
test. to_csv ( './data/validation-table/valid_data.csv', index=False, encoding='utf-8')

TAMBEIN SE CREARON LOS MLTABLE DE CADA UNOS DE LOS DATASET PARA EL ENTRENAMIENTO, PARA QUE AL EL PATCH. 

In [7]:
paths = [{"file": "./data/training-table/train_data.csv"}]
tbl = mltable.from_delimited_files(
    paths= paths,
    delimiter=",",
    header=MLTableHeaders.all_files_same_headers,
    infer_column_types=True,
    include_path_column=False,
    encoding=MLTableFileEncoding.utf8,
)
mltable_folder = "./data/training-table/"
tbl.save(mltable_folder)

In [8]:
paths = [{"file": "./data/validation-table/valid_data.csv"}]
tbl = mltable.from_delimited_files(
    paths= paths,
    delimiter=",",
    header=MLTableHeaders.all_files_same_headers,
    infer_column_types=True,
    include_path_column=False,
    encoding=MLTableFileEncoding.utf8,
)
mltable_folder = "./data/validation-table/"
tbl.save(mltable_folder)

SE PRECEDE A COLOCAR LOS DATASET EN ENTRADAS UTILIZABLES PARA EL MODELO.

In [9]:
training_mltable_path = "./data/training-table/"
validation_mltable_path = "./data/validation-table/"

# Training MLTable defined locally, with local data to be uploaded
my_training_data_input = Input(type=AssetTypes.MLTABLE, path=training_mltable_path)

# Validation MLTable defined locally, with local data to be uploaded
my_validation_data_input = Input(type=AssetTypes.MLTABLE, path=validation_mltable_path)

# <mark>**CREACION DEL CLUSTER PARA LA UTILIZACION EN EL ENTRENAMIENTO DE MODELOS**</mark>

EN ESTE PASO SE ESCOGIO EL"STANDAR_E4DS_V4" YA QUE ESTE CLUSTER ESTA OPTIMIZADO PARA EL USO DE MEMORIA Y PARA DATAS NO TAN GRANDES, CONTANDO CON 4 GB DE RAM, 32GB DE MEMORIA Y UN PROCESADOR INTEL(R) XEON(R) PLATINUM 8272CL CPU @ 2.60GHZ.
SE UTILIZO EL MAXIMO CANTIDAD DE NODOS PERMITIDOS POR ESTE CLUSTER QUE SON 2.

In [10]:
compute_name = "Cpu-Cluster-E4dsv4"

try:
    _ = ml_client.compute.get(compute_name)
    print("Found existing compute target.")
except ResourceNotFoundError:
    print("Creating a new compute target...")
    compute_config = AmlCompute(
        name = compute_name,
        type = "amlcompute",
        size = "Standard_E4ds_v4",
        idle_time_before_scale_down = 120,
        min_instances = 0,
        max_instances = 2,
    )
    ml_client.begin_create_or_update(compute_config).result()

Found existing compute target.


# <mark>**Experimento**</mark>

AQUI CONFIGURAMOS EL EXPERIMENTO QUE RALIZARA UNA TAREA DE CLASIFICACION CON LOS DATAS DE ENTRENAMIENTO Y VALIDACION, CON UNA METRICA BASADA EN LA PRECISION, E UTILIZANDO LA COLUMNA DE SALIDA DE SI BEBE O NO ALCOHOL, SE PROCEDIO A TAMBIEN COLOCAR UN LIMITE DE TIEMPO QUE TRABAJE EL ENTRENAMIENTO DE 30 MINUTOS Y QUE SOLO REALICE HASTA 8 MODELOS CON UN TRABAJO EN PARALELO DE 2.

In [11]:
exp_name = 'clasificacion'
exp_timeout = 30
classification_job = automl.classification(
    compute = compute_name,
    experiment_name = exp_name,
    training_data = my_training_data_input,
    validation_data = my_validation_data_input,
    target_column_name = "DRK_YN",
    primary_metric = "accuracy",
    tags = {"my_custom_tag": "My_custon_value"},
)
classification_job.set_limits(
    timeout_minutes=exp_timeout,
    max_trials=8,
    max_concurrent_trials=2,
    )


LUEGO SE ENVIO EL JOB A AUTOML PARA QUE SE EJECUTE.

In [12]:
returned_job = ml_client.jobs.create_or_update(
    classification_job
)  # submit the job to the backend

print(f"Created job: {returned_job}")

Uploading training-table (15.1 MBs): 100%|██████████| 15098272/15098272 [00:00<00:00, 33903003.75it/s]


Uploading validation-table (3.77 MBs): 100%|██████████| 3774695/3774695 [00:00<00:00, 15098823.20it/s]




Created job: compute: azureml:Cpu-Cluster-E4dsv4
creation_context:
  created_at: '2023-10-28T16:41:31.345756+00:00'
  created_by: maribel atahuichi
  created_by_type: User
display_name: calm_turtle_jyc80gz19c
experiment_name: clasificacion
id: azureml:/subscriptions/fc14670d-c5e7-4c9f-aaa4-11ab881abb5f/resourceGroups/proyecto1-modulo4/providers/Microsoft.MachineLearningServices/workspaces/proyecto1/jobs/calm_turtle_jyc80gz19c
limits:
  enable_early_termination: true
  max_concurrent_trials: 2
  max_cores_per_trial: -1
  max_nodes: 1
  max_trials: 8
  timeout_minutes: 30
  trial_timeout_minutes: 30
log_verbosity: info
name: calm_turtle_jyc80gz19c
outputs: {}
primary_metric: accuracy
properties: {}
resources:
  instance_count: 1
  shm_size: 2g
services:
  Studio:
    endpoint: https://ml.azure.com/runs/calm_turtle_jyc80gz19c?wsid=/subscriptions/fc14670d-c5e7-4c9f-aaa4-11ab881abb5f/resourcegroups/proyecto1-modulo4/workspaces/proyecto1&tid=ce634528-6f22-4a15-a253-f0d189edbe44
  Tracking:
 

COMANDO PARA VER EN QUE ESTADO ESTA EL ENTRENAMIENTO DE MODELOS.

In [13]:
ml_client.jobs.stream(returned_job.name) 

RunId: calm_turtle_jyc80gz19c
Web View: https://ml.azure.com/runs/calm_turtle_jyc80gz19c?wsid=/subscriptions/fc14670d-c5e7-4c9f-aaa4-11ab881abb5f/resourcegroups/proyecto1-modulo4/workspaces/proyecto1

Execution Summary
RunId: calm_turtle_jyc80gz19c
Web View: https://ml.azure.com/runs/calm_turtle_jyc80gz19c?wsid=/subscriptions/fc14670d-c5e7-4c9f-aaa4-11ab881abb5f/resourcegroups/proyecto1-modulo4/workspaces/proyecto1



In [14]:
import mlflow

# Obtain the tracking URL from MLClient
MLFLOW_TRACKING_URI = ml_client.workspaces.get(
    name=ml_client.workspace_name
).mlflow_tracking_uri

print(MLFLOW_TRACKING_URI)

azureml://eastus.api.azureml.ms/mlflow/v1.0/subscriptions/fc14670d-c5e7-4c9f-aaa4-11ab881abb5f/resourceGroups/proyecto1-modulo4/providers/Microsoft.MachineLearningServices/workspaces/proyecto1


In [15]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [16]:
from mlflow.tracking.client import MlflowClient
from mlflow.artifacts import download_artifacts

# Initialize MLFlow client
mlflow_client = MlflowClient()

<mark>**PRECEDEMOS A OBTENER LA INFORMACION DEL JOB**</mark>

In [18]:
job = ml_client.jobs.get(name='calm_turtle_jyc80gz19c')

In [19]:
mlflow_parent_run = mlflow_client.get_run(job.name)

In [20]:
print(mlflow_parent_run)

<Run: data=<RunData: metrics={'AUC_macro': 0.7906512174377998,
 'AUC_micro': 0.7908162421812797,
 'AUC_weighted': 0.7906512174377998,
 'accuracy': 0.7139560688976875,
 'average_precision_score_macro': 0.7864441325664651,
 'average_precision_score_micro': 0.7869097934850179,
 'average_precision_score_weighted': 0.7864488153589011,
 'balanced_accuracy': 0.7139630565438447,
 'f1_score_macro': 0.7139373808007399,
 'f1_score_micro': 0.7139560688976874,
 'f1_score_weighted': 0.7139355149065049,
 'log_loss': 0.5653932738160792,
 'matthews_correlation': 0.42798762054147765,
 'norm_macro_recall': 0.42792611308768946,
 'precision_score_macro': 0.7140245684179836,
 'precision_score_micro': 0.7139560688976875,
 'precision_score_weighted': 0.7140278254531239,
 'recall_score_macro': 0.7139630565438447,
 'recall_score_micro': 0.7139560688976875,
 'recall_score_weighted': 0.7139560688976875,
 'weighted_accuracy': 0.7139490812606313}, params={}, tags={'automl_best_child_run_id': 'calm_turtle_jyc80gz19c

LOS MODELOS QUE UTILIZO AUTOML SON: “VOTINGENSEMBLE”, "STACKENSEMBLE”, “MAXABSSCALER, XGBOOSTCLASSIFIER”, “SPARSENORMALIZER, XGBOOSTCLASSIFIER”, “MAXABSSCALER, LIGHTGBM”, “MAXABSSCALER, EXTREMERANDOMTREES”, EL MAXABSSCALER LO UTILIZO 3 VECES PERO CON DIFERENTES HIPERPARAMETROS.
EL MEJOR MODELO QUE SACO ES EL DE VOTINGENSEBLE CON UNA PRESICION DEL 0.7167.
LOS HIPERPARAMETROS UTILIZADOS POR AUTOML SON:“algorithm : ['LightGBM', 'XGBoostClassifier', 'LightGBM', 'XGBoostClassifier']”, “min_data_in_leaf : 20”, “tree_method : auto”,  “boosting_type : gbdt”, “colsample_bytree : 0.6933333333333332”, “learning_rate : 0.09473736842105263”, “max_bin : 110, max_depth : 8, +8”, “booster : gbtree, colsample_bytree : 0.7, eta : 0.01, gamma : 0.01, max_depth : 7, +7”, ”boosting_type : gbdt, colsample_bytree : 0.5944444444444444, learning_rate : 0.06316157894736842, max_bin : 260, max_depth : 6, +8”, “bootstrap : true, class_weight : balanced, criterion : Gini, max_features : sqrt, min_samples_leaf : 0.01, +3” 


<mark>**AQUI SACAMOS EL MEJOR MODELO QUE SE ENTRENO**</mark>

In [21]:
best_child_run_id = mlflow_parent_run.data.tags["automl_best_child_run_id"]

In [22]:
best_run = mlflow_client.get_run(best_child_run_id)

In [23]:
best_run

<Run: data=<RunData: metrics={'AUC_macro': 0.7906512174377998,
 'AUC_micro': 0.7908162421812797,
 'AUC_weighted': 0.7906512174377998,
 'accuracy': 0.7139560688976875,
 'average_precision_score_macro': 0.7864441325664651,
 'average_precision_score_micro': 0.7869097934850179,
 'average_precision_score_weighted': 0.7864488153589011,
 'balanced_accuracy': 0.7139630565438447,
 'f1_score_macro': 0.7139373808007399,
 'f1_score_micro': 0.7139560688976874,
 'f1_score_weighted': 0.7139355149065049,
 'log_loss': 0.5653932738160792,
 'matthews_correlation': 0.42798762054147765,
 'norm_macro_recall': 0.42792611308768946,
 'precision_score_macro': 0.7140245684179836,
 'precision_score_micro': 0.7139560688976875,
 'precision_score_weighted': 0.7140278254531239,
 'recall_score_macro': 0.7139630565438447,
 'recall_score_micro': 0.7139560688976875,
 'recall_score_weighted': 0.7139560688976875,
 'weighted_accuracy': 0.7139490812606313}, params={}, tags={'mlflow.parentRunId': 'calm_turtle_jyc80gz19c',
 'm

<mark>**SE PROCEDE A DESCARGAR LOS ARTEFACTOS**</mark>

In [24]:
import os

# Create local folder
local_dir = "./artifact_downloads"
if not os.path.exists(local_dir):
    os.mkdir(local_dir)
# Download run's artifacts/outputs
local_path = download_artifacts(
    run_id=best_run.info.run_id, artifact_path="outputs", dst_path=local_dir
)

In [25]:
# Show the contents of the MLFlow model folder
os.listdir("./artifact_downloads/outputs/mlflow-model")

['conda.yaml', 'MLmodel', 'model.pkl', 'python_env.yaml', 'requirements.txt']

<mark>**SE PROCEDE A CREAR EL ENDPOINT PARA REALIZAR EL DEPLOY**</mark>

In [22]:
# import required libraries
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
    ProbeSettings,
)

# Creating a unique endpoint name with current datetime to avoid conflicts
import datetime

online_endpoint_name = "clasificacion" + datetime.datetime.now().strftime("%m%d%H%M%f")

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="this is a sample online endpoint for deploying model",
    auth_mode="key",
    tags={"foo": "bar"},
)
print(online_endpoint_name)

clasificacion10222314412911


In [23]:
ml_client.begin_create_or_update(endpoint).result()

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://clasificacion10222314412911.eastus.inference.ml.azure.com/score', 'openapi_uri': 'https://clasificacion10222314412911.eastus.inference.ml.azure.com/swagger.json', 'name': 'clasificacion10222314412911', 'description': 'this is a sample online endpoint for deploying model', 'tags': {'foo': 'bar'}, 'properties': {'azureml.onlineendpointid': '/subscriptions/ed3278d5-430f-46e8-8964-83e5db2c41c4/resourcegroups/proyecto-1-mod-1/providers/microsoft.machinelearningservices/workspaces/proyecto-1-now/onlineendpoints/clasificacion10222314412911', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/ed3278d5-430f-46e8-8964-83e5db2c41c4/providers/Microsoft.MachineLearningServices/locations/eastus/mfeOperationsStatus/oe:9ded5618-afa1-433d-88e3-a22704c1fc88:70baf5d1-9064-4a3c-b2d3-ce5332c72586?api-version=2022-02-01-preview'}, 'print_as_yaml': True, 'id': '/subscriptio

In [24]:
model_name = "classification-model"
model = Model(
    path=f"azureml://jobs/{best_run.info.run_id}/outputs/artifacts/outputs/mlflow-model/",
    name=model_name,
    description="my sample classification task",
    type=AssetTypes.MLFLOW_MODEL,
)
registered_model = ml_client.models.create_or_update(model)

In [36]:
deployment = ManagedOnlineDeployment(
    name="classification-deployed",
    endpoint_name=online_endpoint_name,
    model=registered_model.id,
    instance_type="Standard_E4s_v3",
    instance_count=1)

ml_client.online_deployments.begin_create_or_update(deployment).result()

Check: endpoint clasificacion10222314412911 exists


..............................................................................................

ManagedOnlineDeployment({'private_network_connection': None, 'provisioning_state': 'Succeeded', 'endpoint_name': 'clasificacion10222314412911', 'type': 'Managed', 'name': 'classification-deployed', 'description': None, 'tags': {}, 'properties': {'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/ed3278d5-430f-46e8-8964-83e5db2c41c4/providers/Microsoft.MachineLearningServices/locations/eastus/mfeOperationsStatus/od:9ded5618-afa1-433d-88e3-a22704c1fc88:c9f2df7c-c135-4307-af06-6a0031437016?api-version=2023-04-01-preview'}, 'print_as_yaml': True, 'id': '/subscriptions/ed3278d5-430f-46e8-8964-83e5db2c41c4/resourceGroups/proyecto-1-mod-1/providers/Microsoft.MachineLearningServices/workspaces/proyecto-1-now/onlineEndpoints/clasificacion10222314412911/deployments/classification-deployed', 'Resource__source_path': None, 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/eddyatahuichizabalaga1/code/Users/eddyatahuichizabalaga', 'creation_context': None, 'serialize':

SE UTILIZO UNA INSTANCIA STANDAD_E4S_V3 YA QUE ESTAS SI ESTAN HABILITADAS PARA REALIZAR ENDPOINT, TAMBIEN POR QUE ESTA OPTIMIZADO PARA USO DE MEMORIA.

# <mark>**PREDICCION**</mark>

In [1]:
import json

request_json = {
    "input_data": {
        "columns": ['sex', 'height', 'weight', 'waistline', 'DBP', 'triglyceride',	'hemoglobin', 'serum_creatinine',	'SGOT_ALT',	'gamma_GTP',	'SMK_stat_type_cd'],
        "data": [{'sex':0, 'height':155, 'weight':70, 'waistline':780, 'DBP':820, 'triglyceride':210, 'hemoglobin':133, 'serum_creatinine':6, 'SGOT_ALT':60, 'gamma_GTP':120, 'SMK_stat_type_cd':1}],
    }
}
request_file_name = "sample_request_data.json"
with open(request_file_name, "w") as request_file:
    json.dump(request_json, request_file)

In [89]:
resp = ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name=deployment.name,
    request_file=request_file_name,
)

VALORES DE 0 o 1 SI TOMO ALCOHOL.

In [90]:
print(resp)

[0]
